In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 36519157
paper_name = 'cao_wang_2022' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [25]:
original_data1 = pd.read_csv('raw_data/sensitive.txt', header=None, names=['orfs'])
original_data2 = pd.read_csv('raw_data/resistant.txt', header=None, names=['orfs'])

In [26]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 45 x 1
Original data dimensions: 10 x 1


In [27]:
original_data1['orf'] = original_data1['orfs'].astype(str)
original_data2['orf'] = original_data2['orfs'].astype(str)

In [28]:
# Eliminate all white spaces & capitalize
original_data1['orf'] = clean_orf(original_data1['orf'])
original_data2['orf'] = clean_orf(original_data2['orf'])

In [29]:
# Translate to ORFs 
original_data1['orf'] = translate_sc(original_data1['orf'], to='orf')
original_data2['orf'] = translate_sc(original_data2['orf'], to='orf')

In [30]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orf'])
print(original_data1.loc[~t,])

# Make sure everything translated ok
t = looks_like_orf(original_data2['orf'])
print(original_data2.loc[~t,])

Empty DataFrame
Columns: [orfs, orf]
Index: []
Empty DataFrame
Columns: [orfs, orf]
Index: []


In [31]:
original_data1['data'] = -1
original_data2['data'] = 1
original_data = pd.concat([original_data1, original_data2], axis=0)

In [32]:
original_data.set_index('orf', inplace=True)

In [33]:
original_data = original_data[['data']].copy()

In [34]:
original_data = original_data.groupby(original_data.index).mean()

In [35]:
original_data.shape

(55, 1)

# Prepare the final dataset

In [36]:
data = original_data.copy()

In [37]:
dataset_ids = [22238]
datasets = datasets.reindex(index=dataset_ids)

In [38]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [39]:
data.head()

dataset_id,22238
data_type,value
orf,
YAL013W,1
YBL102W,1
YBR059C,-1
YBR103W,-1
YBR126C,-1


## Subset to the genes currently in SGD

In [40]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [41]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22238
,data_type,value
gene_id,orf,
11,YAL013W,1
190,YBL102W,1
255,YBR059C,-1
297,YBR103W,-1
320,YBR126C,-1


# Normalize

In [42]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [43]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [44]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22238          
data_type       value    valuez
gene_id orf                    
11      YAL013W     1  9.213693
190     YBL102W     1  9.213693
255     YBR059C    -1 -9.029419
297     YBR103W    -1 -9.029419
320     YBR126C    -1 -9.029419

# Print out

In [45]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')